## $k$-nearest neighbor classifier with $k$ chosen by cross validation

In [8]:
#function U=knn(X,Y,T,ks)
#m=size(T,2); D2=repmat(sum(T.^2,1),[size(X,2) 1]);
#D2=D2+repmat(sum(X.^2,1)',[1 m])-2*X'*T; [dum,z]=sort(D2,1);
#for i=1:length(ks)
#k=ks(i);
#for j=1:m
#Z=sort(Y(z(1:k,j))); g=find([1 Z(1:end-1)~=Z(2:end)]);
#[dum,a]= max([g(2:end) k+1]-g); U(i,j)=Z(g(a));
#end, end

In [9]:
#load digit.mat X T; [d,m,c]=size(X); X=reshape(X,[d m*c]);
#Y=reshape(repmat([1:c],[m 1]),[1 m*c]);
#ks=[1:10]; t=5; v=mod(randperm(m*c),t)+1;
#for i=1:t
#Yh=knn(X(:,v~=i),Y(v~=i),X(:,v==i),ks);
#s(i,:)=mean(Yh~=repmat(Y(v==i),[length(ks) 1]),2);
#end
#[dum,a]=min(mean(s)); k=ks(a); [d,r,c]=size(T);
#T=reshape(T,[d r*c]); U=reshape(knn(X,Y,T,k),[r c]);
#for i=1:c, C(:,i)=sum(U==i); end, C, sum(diag(C))/sum(sum(C))

In [10]:
import numpy as np
#import numpy.matlib
from scipy import io

In [18]:
def knn(X,Y,T,ks):
    m=np.size(T,1);D2=np.kron(np.ones((np.size(X,1),1)), np.sum(T*T,0));
    D2=D2+np.kron(np.ones((m,1)), np.transpose(np.sum(X*X,0))).T-2*np.transpose(X)@T;
    dum=np.sort(D2,0);z=np.argsort(D2,0);
    U=np.zeros((len(ks),m));
    for i in range(1,len(ks)+1):
        k=ks[i-1];
        for j in range(1,m+1):
            Z=np.sort(Y[z[0:k,j-1]]);
            g=np.nonzero(np.insert([1],0,Z[1:len(Z)-1]!=Z[2:len(Z)]))[0];
            dum=np.max(np.append(g[1:len(g)],[k+1])-g);a=np.argmax(np.append(g[1:len(g)],[k+1])-g)
            U[i-1,j-1]=Z[g[a]];
    return U

In [60]:
mat = io.loadmat('digit.mat');
X=mat['X'];
X=X[:,0:399,:];
T=mat['T'];
d=np.size(X,0);m=np.size(X,1);c=np.size(X,2);
X=X.reshape(d,m*c,order='F').copy();
Y=np.kron(np.ones((m,1)),range(1,c+1)).reshape(m*c,order='F')
ks=range(1,10+1);t=5; v=np.mod(np.random.permutation(m*c)+1,t)+1;

In [61]:
s=np.zeros((t,len(ks)));
for i in range(1,t+1):
    Yh=knn(X[:,v!=i],Y[v!=i],X[:,v==i],ks);
    s[i-1,:]=np.mean(Yh!=np.kron(np.ones((len(ks),1)), Y[v==i]),axis=1);
dum=np.min(np.mean(s));a=np.argmin(np.mean(s));k=ks[a];[d,r,c]=T.shape;
T=T.reshape(d,r*c,order='F').copy();U=(knn(X,Y,T,[k])).reshape(r,c,order='F').copy();
C=np.zeros((c,c));
for i in range(1,c+1):
    C[:,i-1]=np.sum(U==i,0);
print("Confusion matrix:\n%s" % C)

Confusion matrix:
[[200.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0. 192.   2.   0.   0.   0.   2.   3.   1.   0.]
 [  0.   0. 195.   0.   3.   0.   0.   1.   1.   0.]
 [  0.   0.   0. 190.   1.   2.   0.   0.   7.   0.]
 [  0.   2.   4.   0. 188.   0.   1.   1.   2.   2.]
 [  0.   2.   0.   0.   2. 195.   0.   0.   0.   1.]
 [  0.   0.   1.   2.   0.   0. 193.   1.   3.   0.]
 [  0.   1.   5.   0.   1.   0.   0. 188.   2.   3.]
 [  0.   0.   0.   2.   0.   0.   1.   1. 196.   0.]
 [  0.   1.   1.   0.   0.   1.   0.   0.   0. 197.]]


In [62]:
np.sum(np.diag(C),0)/np.sum(np.sum(C))

0.967